## The classic strategy

### Inputs
- stock price
- bond price/CDS upfront
- instrument recovery
- debt per share
- interest rate term structure and expected maturity (bonds only)
- trading grids: interactive inputs, determined after reviewing outputs from the model

### Parameters
- global recovery: $\bar{L}$
- global recovery standard deviation: $\lambda$
- reference stock price: $S_0$ - **not used in core calculation**
- **reference stock vol not an input**

### Outputs
- Implied asset vol: realtime, pre/post-override time-series, mis-pricing time-series, IV vs spread (scatter plot)
- The Greeks: see below and their corresponding PnL
    - Delta: vs its empirical counterpart
- bid and ask price/upfront: a function of both model output and the trading grid.

### Greek expansion

\begin{align}
dB = \underbrace{\frac{\partial B}{\partial A}dA}_\text{delta} + \underbrace{\frac{\partial B}{\partial \sigma_A}d\sigma_A}_\text{vega} + \underbrace{\frac{1}{2}\frac{\partial^2B}{\partial A^2}A^2\left(<\frac{dA}{A}, \frac{dA}{A}> - \sigma_A^2\right)dt}_\text{gamma} + \underbrace{\frac{\partial^2 B}{\partial A\partial\sigma_A}<dA, d\sigma_A>}_\text{vanna/cross risk} + \text{residual} 
\end{align}

Notes
- The above **greek expansion** is due to Black-Scholes. Technically we should do the same using the CGM dynamics and that it is a barrier option, but the above is good in providing some basic intuitions: **just be very careful about what you want to map!**
- The above framework helps draw inspirations from **eq vol market making**. For instance, in the CGM model, there is actually **no explicit mention of bonds/CDS being puts in CGM**, but on an intuitive level bonds/CDS are mapped to puts.
- In a structural model's perspective, **long credit is like shorting puts**, while **longing converts is like risk reversals: selling puts + longing calls**.

### PnL attribution
- The pnl attribution above up the residual term accounts for around $95\%$ of the risks.
- Main contribution components are delta and vega (pull), which is normally ~$90\%$. For healthy distressed names, gamma (drift) and vanna (cross risks) could be sizable ($20-30\%$).

### vega
- Fundamental research on direction of $\sigma$. 
    - In the long run, firms tend to maintain a stable leverage, either to its long-run mean or to the sector average. This is either due to company organic growth and natural delever, or by explicit management choice in order to maintain a rating target.
- Mean-reverting signals historically and cross-sectionally. 
    - In the medium and short run, there are unwind or reversal of CTA/asset managers, or transient yet structural flows such as ETF arb, which can cause the relative price to deviate and create flow in-efficiencies. Unwind of large positions can cause dislocations as well.
    - There are also other parties arbitraging this to keep the price in place, or mean-reverting.
- Information from eq vol surface <font color=red>**(work-in-future)** </font>
    - For example, eq option volatility tends to jump around equities events such as earnings, which may or may not be a good entry point. So one needs to **smooth out the implied eq options vol before mapping it to the implied asset vol**.
    - **Which eq implied vol to choose**: there is a whole vol surface to choose from!
        - The current procedure is to use empirical delta to determine the strike and credit duration to determine the expiry.
            - The procedure for strikes fits the intuition that for safe names, use OTM put vol; while for distressed names, use ATM vol, but the choice of empirical delta, which is $\frac{\partial B/B}{\partial S/S}$, is hand-waving.
            - The proxy by credit duration could be problematic too: the expiry of a 1y barrier option does not become 1w just because stock price is very close to the barrier - its implied eq vol will still be marked at the 1y point.
        - A more consistent way is to map the credit instrument to DOOM puts, but that may require the eq vol surface to extend to long expiries, which are constructed usually using credit information. So we run into a chicken-and-egg problem.
    - Maybe there should be a **different mapping between asset vol and equity vol**, not the one using leverage as specified in the CGM model. The CGM model in a sense is just using a prior result based on hand-wavingly assumption, which could be inconsistent. At the very least, the relationship should be intuitively a function of input recovery.
    - Is it contradictory that for equity options, implied vol is negatively correlated with equity prices, while in the CGM model, IV increases as equity price increases (as bond becomes relatively cheaper)?
        - This is confusing two different phenomena, the former is an empirical fact, while the latter is an artifact of keeping bond price unchanged in the CGM model to back out IV.
        - In fact, consider a put. If we keep the put price constant in the same way as done in the CGM model, implied equity vol corresponding to the put should increase as well.
- From an [interview](https://www.evernote.com/shard/s191/nl/21353936/a225b122-a50d-a6ba-c344-7aedefbab52b?title=Boaz%20Weinstein%20Reveals%20the%20Secret%20Trade%20That%20Helped%20Make%20Him%20Famous), Boaz Weinstein reveals that a diversification his fund has for low-vol environment is investing in discounted closed-end funds.

### delta
- Matching empirical delta to ascertain correct model inputs (recovery and debt-per-share), using **Bayesian regression (need formula here)**; see this [notebook](../bayesian-regression-emp-delta/bayesian-regression-emp-delta.ipynb)
- Use vanna, or the cross term to adjust delta.
    - The intuition is, for cases where vanna is large, **do not hedge using the model delta at the current IV, but the future expected IV (or your view)**
    - Try a thought experiment of long bond. We are expecting the IV to decrease, as such in future, the short put due to long bond is less ITM, and thus the delta should be adjusted lower from the model value from current IV. This is **consistent with the observed problem** that model delta is always too heavy for somewhat distressed, long maturity IG names (e.g. ORCL).
    - See this [notebook](../vanna-adjustment/vanna-adjustment.ipynb) for more details.

### gamma/carry
- Simulation to produce a margin of errors

### rho
For bonds (and for CDS with big MtM value) there is also an interest rate sensitivity. 
- Currently this is dealt with using central team's callable bond model and obtain an expected maturity, take the **rolling 30-day average** of that and **create a synthetic bullet bond** out of that maturity. Then overlay that synthetic bond with the credit component produced by the CGM model. 
- The above is one way to insulate the interest component, but entirely sure about how good it is. Also this is **different from what is done in basis**.
- Relatedly, interest rate hedge is done via long/short Treasury, therefore **only account for linear interest-rate risks**. Note that is done on aggregate for the whole d/e portfolio. 
- Ideally, for IG bonds, we can probably safely use swaptions. For HY bond, a model is needed, **calibrating the true call prices of the issuer**, and thus a bit different swaption hedges from what would be by using the nominal call prices.

## Flow Metrics to Monitor

- From an [interview](https://www.evernote.com/shard/s191/nl/21353936/a225b122-a50d-a6ba-c344-7aedefbab52b?title=Boaz%20Weinstein%20Reveals%20the%20Secret%20Trade%20That%20Helped%20Make%20Him%20Famous), Boaz described his trade with Morgan Stanley. It would seem that Boaz not only looks at Net Open Interest, but also able to estimate how much is due to one counterparty: don't know how he does it.
- In general, flow metrics are important to look at. A few examples are the DTCC net interest, ICE volumes etc.

## Strategy extension

### Basket Instruments

#### Two definitions of basket mispx
- 1st definition of basket mispx: **weighted average of single-name mispx**
    \begin{align}
    \frac{\sum_{n=1}^N w_n \cdot \nu_{n, t}(\sigma_{n, t} - \bar{\sigma_{n}})}{\sum_{n=1}^N w_n},
    \end{align}
    where
    - $w_n$: generic single-name weight, fixed through time, i.e. not depending on $t$;
    - $\nu_{n, t}$: single-name vega at time $t$;
    - $\sigma_{n, t}$: single-name IV at time $t$;
    - $\bar{\sigma_n}$: 6m avg single-name IV.
- 2nd definition of basket mispx: **basket vega $\times$ basket IV**

**Motivation**: meaningful definition of basket vega and basket IV should be such that the day-over-day change of the 1st way equals the 2nd way, i.e. the pnl (mispx) should be able to attribute to vega times IV changes.

#### One solution of basket IV and vega
- basket IV
    \begin{align}
    \frac{\sum_{n=1}^N w_n \cdot \nu_{n, t}\sigma_{n, t}}{\sum_{n=1}^N w_n \cdot\nu_{n, t}}
    \end{align}
- basket vega
    \begin{align}
    \frac{\sum_{n=1}^N w_n \cdot \nu_{n, t}}{\sum_{n=1}^N w_n}
    \end{align}

**Check**: it is straightforward to see the day-over-day change of 1st, and that of the 2nd way using the above definition, are both
\begin{align}
\frac{\sum_{n=1}^N w_n \cdot \nu_{n, t}\Delta\sigma_{n, t}}{\sum_{n=1}^N w_n},
\end{align}
where $\Delta\sigma_{n, t}$ is the day-over-day change of IV. The above holds because
- the term $\sum_{n=1}^N w_n \cdot\nu_{n, t}$ cancels out in basket IV $\times$ basket vega;
- $\Delta \bar{\sigma_n}$ should be negligible numerically.

#### Why else should the above definition makes sense?
- The definition of basket IV makes **good analogy to how index spread is defined**; to recall, the latter is 
    \begin{align}
    \frac{\sum_{n=1}^N w_n \cdot SpreadDuration_{n, t}\cdot Spread_{n, t}}{\sum_{n=1}^N w_n \cdot SpreadDuration_{n, t}}
    \end{align}
- Empirically, single-name IV tends to **move in parallel rather than in proportion**, so it makes sense that the basket vega, a risk sensitivity metric of the basket, is just the weighted average of single-name vega risk sensitivity.

#### Skew adjusted CDS baskets

For CDS custom basket, we calculated basket IV adjusted for skew - we want to ascertain relative value when accounting for the skew against CDX. 

The methodology is $Skew = IndexSpread - nav$, where $nav$ is aggregated quoted spread from the basket. Then each single name's IV is adjusted to $SkewAdjustedIV_n = IV_n + Skew\;/\;SpreadChangePer1ptIV$. After that, $SkewAdjustedIV_n$ is aggregated, now using fixed weight times vega: $w_n\cdot vega_n$.

- $nav$ is currently $\sum w_n \cdot QuotedSpread_n\;/\;\sum w_n$ in the code, but I think it should be $\sum w_n \cdot Duration_n\cdot QuotedSpread_n\;/\;\sum w_n \cdot Duration_n$ to be more inline with ETF definition.
- The skew above is defined as **level change**, but I think probably **percentage change** makes more sense. That is, we adjust each name's spread by the same proportion so that the new nav equals the index spread.
    - Say if the skew is 50bps, while a single name rarely quotes above 20bps, then the level adjustment will arrive at an adjusted IV that does not make sense to this single name. (Skew being 50bps is probably unrealistic, but you get the idea). 
    - Another benefit is that proportion change can prevent $SkewAdjustedIV_n$ to drop below 0.

<font color=red> **Open questions** </font >

- BW is looking into index relative value and index option relative value: **credit index options vs eq index options. Don't know how he does it**.
- Probably it is beneficial to look at this time-series: $\frac{\sum w_n\cdot \Delta B_n}{\sum w_n}$ regressed on $\frac{\sum w_n\cdot DeltaPct_n\cdot \Delta S_n}{\sum w_n\cdot DeltaPct_n\cdot S_n}$
    - It is the basket bond portfolio price change, on basket eq portfolio percentage change.
    - It is closer to the single name counterpart of $\Delta B_n$ on $(\Delta S_n / S_n)$, rather than weighting average single-name emprical betas.
    - I would expect the $R^2$ of the regression above to be high since the noise in single name cancels out each other, thereby providing new insights about the macro hedge ratios, though I am still less clear on how to utilize it.
- How do we account for the **skew** between the macro instrument and the basket of individual instruments?
    - The above skew adjustment on CDX index could be one way, but it does not quite take into account **why index basis exists**, simply allocating the skew onto the constituents may not be a good solution.
    - Can we **back out the IV of say CDX HY index and engage in some kind of dispersion trading**, e.g. see Section 6.3 of [< Trading Volatility >](https://www.evernote.com/shard/s191/nl/21353936/8de185c0-799e-426a-c67c-996e158f20eb?title=Trading%20Volatility,%20Correlation,%20Term%20Structure%20and%20Skew)

### EM hard currency debt vs local currency debt

Basically, treat EM hard currency debt as a corp's debt, while local currency debt as a corp's equities. The reasons we can apply this mapping are as follows.
- It is observed that sovereign hard currency debt is **senior in payment order**. Indeed, unless under peculiar circumstances, sovereigns will try to repay foreign-currency debt, while use the printing press to satisfy local currency debt needs.
- By 'local currency debt', we also tally the $M_0$, which makes local currency debt the 'equities' rather than just a layer of junior debt.

The goal of this strategy is to obtain uncorrelated alpha with respect to the classic strategy.

#### What are the economic rational that the relative price is mean-reverting?
- EM sovereigns will try to **keep leverage (e.g. foreign debt over GDP, in US dollar term) stable**.
- There maybe **lead-lag relationships** between local and hard-ccy debts.
- Sovereign credit risk are the main driver of prices for the two, which means **mostly the two should move together**. Though FX risk might diverge the two, under normal circumstances, **FX rate is mean-reverint as well**. In other words, once FX risk is accounted for, by law of one price, local ccy debt and hard ccy debt should have discounted using one discount curve.

#### What procedures can carry over from the classical debt-equity strategy, maybe with some minor adjustments?

- From a modeling perspective, just like equities can be viewed as a bundle of credit risk + equities specific risks which can be handled by structural model, **local currency debt can also be viewed as credit risk + exchange risk and similarly be handled by the structural model**.
- One may **use FX vol for screening**, just like using equities vol for screening in the classical strategy: some modeling work may be needed to map between FX vol and asset vol.
- There could be **fundamental indicators** to watch
    - Though there is a **preference of sovereign to issue local currency debt** when they can, corps prefer issuing debts over equities. Moreover, sovereigns do not typically issue local currency debt to retire hard currency debt.
    - **Monetary policy**: not sure about the direction though - when local interest rate tightens, local ccy bond price drops, but exchange rate goes up. Probably big monetary policy shift may create price dislocation though.
    - **Fiscal balances**, **DDIS** and **foreign reserve gaps** could determine issuance of local vs hard currency debt.
- The delta hedge will be like long hard currency debt and short local currency/hard currency FX pair.
    - Even without any model, this long-short relationship makes intuitive sense: there are fundamental or economic reasons why the two should move together. But it could be a bit **weird on first thought**: we are trading hard currency debt, with no explicit FX exposure, yet we are hedging with FX.
    - Thus model is just **a consistent way to come up with signals with relative value**.
    - As such, **replace in the input the stock price to be the FX price** in this variant. 
    - To **compute the hedge ratio, the CGM model will need to be adjusted**, for instance, the limit in (2.22) is no longer 1, but perhaps the hard currency value weight of $M0$ amongst the local debt.

### Equity gamma trade vs DOOM puts

The strategy is to use DOOM put or put spreads to syntheticate a CDS, then we can perform rv trade just as the classical case.
- Given that Boaz Weistein was doing arbitrage between puts and CDS, it is likely (remains to be tested) that the replication relation should hold: that is, if there is a trade between CDS and equities, there should be a trade between DOOM and equities.
    - But of course, DOOMs could be of much smaller expirary than CDS. That makes this trade particularly suitable for short-tenor view expression.
    - Relatedly, the motivation of this trade is to express views quickly, rather than waiting on negotiations of CDS.
- IV and delta can be produced by CGM model: **replace credit price by put spread price**.
- One can use the usual historical and cross-sectional metrics to ascertain rv directions: this part stays the same
- For a full structural model on puts, probably recovery is no longer an input, but DPS still is. One can compute a market-implied DPS by matching cgm delta with the one that would neutralize equities delta.
    - Note that in [Carr and Wu (2011)](https://www.evernote.com/shard/s191/nl/21353936/8423e66b-c011-2c8c-3338-fec686479077?title=Carr%20and%20Wu%20(2011)), it is posited that DOOM puts do not have delta and vega. This is due to their assumption that default corridors are never entered unless the firm is in default. Apparently this is not true in practice, and a parametric model (such as CGM) rather than their non-parametric approach is more appropriate.
- Put spread is probably more appropriate than just put or ratio put spread, if we consider both upfront cost and the protection when equities gap down.
    - Put spread also better monetizes the vol skew; on that front, more understanding on the vol skew is probably warranted.
- Maybe beneficial to look at 'pinning' as well, i.e. situations where there is significant OI around ATM strikes, which indicates large delta hedging activities of broker-dealers. This can have implications on sizing the trade.
- On choosing DOOM puts from [Carr and Wu (2011)](https://www.evernote.com/shard/s191/nl/21353936/8423e66b-c011-2c8c-3338-fec686479077?title=Carr%20and%20Wu%20(2011))
    - OTM: strike price less than $\$5$ and $|\Delta|\leq 15\%$. Note that these two numbers are probably chosen arbitrarily, or just so that there is enough data.
    - expiry: greater than 1 year. But probably we can afford to do near-dates for liquidity. Merton's or CGM does not really have any constraint on tenor after all.

## Old, premitive description on cap structure arbitrage, saved just in case

- **Capital Structure Arbitrage** (credit-equity can be viewed as a special case - take a look at the function *ECCG < GO >*)

> **Expressing views** A capital structure arbitrage portfolio manager considers all securities of a company (debt and equity) for possible mispricing or relative mispricing. They view the securities by payment priority against current cash and future earnings potential. They assess value by ascertaining probabilistic ability to pay debt obligations as well as potential realized equity valuation. Capital structure arbitrage is an investment strategy that is flexible and broad enough in scope to accommodate opportunities that arise throughout the credit cycle, by going up or down the cap structure at different times of the credit cycle.

> **Implementation Details** 

>> The classical trade is probably long-short equity/equity derivative vs CDS spread. In a model-driven way, the [structural model](../models/structural-credit-models.ipynb) is used to calibrate the asset volatility, survival probability and synthetic CDS spread, which can be used to compare to actual observed CDS spread. Some studies of this strategy can be found in these [papers](https://www.evernote.com/shard/s191/nl/21353936/f718aa0d-09b4-434d-827f-21e6abd9d96f).

>> The simplest implementation of this trade is to long on one rank and short the other rank, e.g. short company stock and long debt, either by long the bonds or sell protection.

>> Each of these securities in each layer can be expected to have unique volatility characteristics. The more junior a security, the more likely it will be relatively more volatile. The portfolio manager must consider this volatility when positioning long and short as this will affect the overall P&L volatility exposure to their portfolio. 

>> The portfolio manager must evaluate the covenants of outstanding debt and ascertain payment priority, which can be complicated. The most difficult layer of complication comes from companies with multiple subsidiaries. Subsidiaries and holding companies may have cross-liabilities and not be entirely self-reliant. 

>> Often the capital structure arbitrage portfolio can be concentrated by sector as industry dynamics can affect multiple companies at once in a way that creates opportunity uniquely to that sector.

> **Risks and Hedges** 

>> The long and short legs of the trade can both go against you. 
     
   >>> For instance, in 2005, Boaz Weinstein's GM trade involves shorting the stock and selling bond protection of GM (equivalently longing the bond). But in the summar, the trade got burned when there was a rumor of acquisition which pumps up stock price, while at the same time that GM was downgraded which decreases the value of the CDS. 
   
>> Even if the two legs do not diverge further, they can still stay diverged long enough so that the long-short positions bleed due to carry cost

>> A good credit model is required - think the pre-Crisis credit correlation modeling.

>> Hedging in this strategy tends to be company specific, whether on a junior or senior security basis. However, it is usual to see industry level hedges or even index based ones in this strategy. 

>> Capital structure arbitrage portfolios can have a tendency to be net long for the company specific positions. 

>> Other more generalized hedging techniques may often make sense to deploy in order to mitigate overall market beta/duration exposures. In a sense, the risk that we want to be exposed to is the idio correlation of different instruments under the same issuer/parent; a statistical study that could be useful in this regard is this [paper](https://www.evernote.com/shard/s191/nl/21353936/149ccd7c-70e9-c122-6287-135407f102ed?title=Bloomberg%20Fixed%20Income%20Risk%20Model%20-%20Enhanced%20Model%20of%20Idiosyncratic%20Return%20Correlation).